# Import

In [2]:
import os
from os.path import abspath
import nilearn as nl
from nilearn.image import load_img, math_img
import numpy as np
import nibabel as nib
from fsl.data.image import Image
from fsl.utils.image.resample import resample
from fsl.utils.image.resample import resampleToPixdims

/Users/dima/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Set paths and define variables

In [3]:
data_dir = abspath('../original')
template = nl.image.load_img("template.nii.gz")
affine_matrix = template.affine
affine_shape = template.shape

# Check shapes

In [ ]:
myimg = Image('/Users/dima/Desktop/caruscloud/Bulbus/OB_data_from_Dima/original/Anosmie_001_1952_m.nii.gz')
pixdim = myimg.pixdim[2]
pixdim

In [ ]:
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".nii.gz"):
            img = Image(os.path.join(root, file))
            shape = img.shape
            print(file, shape)
            #print(file, shape)
            #if shape[2] < 32:
            #    print('Shape smaller than 32    ' + file, shape)
            #if shape[2] > 32:
            #    print('Shape greater than 32    ' + file, shape)
            #else:
            #    print('Shape = 32               ' + file)

# Resample and interpolate 

In [4]:
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".nii.gz"):
            fname = file.split(".")[0]
            #img = nl.image.load_img(os.path.join(root, file))
            myimg = Image(os.path.join(root, file))
            shape = int(myimg.shape[2])
            resampled_img = resample(myimg, newShape = (1200, 1200, shape), order=3)
            nifti = nib.Nifti1Image(resampled_img[0], affine=resampled_img[1])
            if not os.path.exists(abspath('../resampled/%s' %fname)):
                os.makedirs(abspath('../resampled/%s' %fname))
            nifti.to_filename(abspath('../resampled/%s/r_%s.nii.gz' %(fname, fname)))
            print(fname)

Anosmie_018_1991_w
Anosmie_104_1959_w
Anosmie_082_1965_w
Anosmie_028_1965_w
Anosmie_020_1988_w
Anosmie_100_1958_w
Anosmie_019_1988_w
Anosmie_105_1941_w
Anosmie_042_1940_w
Anosmie_061_1964_w
Anosmie_012_1973_w
Anosmie_078_1957_w
Anosmie_084_1954_w
Anosmie_024_1967_m
Anosmie_070_1985_m
Anosmie_062_1965_w
Anosmie_022_1940_w
Anosmie_074_1960_m
Anosmie_064_1961_w
Anosmie_063_1985_m
Anosmie_032_1987_w
Anosmie_053_1952_w
Anosmie_086_1964_m
Anosmie_102_1963_m
Anosmie_056_1940_w
Anosmie_029_1989_w
Anosmie_091_1957_m
Anosmie_043_1961_m
Anosmie_090_1966_w
Anosmie_016_1987_w
Anosmie_047_1943_w
Anosmie_076_1969_m
Anosmie_004_1959_m
Anosmie_097_1983_w
Anosmie_072_1990_m
Anosmie_005_1941_w
Anosmie_002_1963_w
Anosmie_017_1982_w
Anosmie_037_1960_w
Anosmie_045_1950_w
Anosmie_060_1966_m
Anosmie_069_1992_m
Anosmie_034_1943_w
Anosmie_023_1960_m
Anosmie_057_1955_m
Anosmie_083_1952_m
Anosmie_073_1969_w
Anosmie_087_1962_w
Anosmie_014_1963_w
Anosmie_021_1949_m
Anosmie_051_1999_m
Anosmie_025_1976_m
Anosmie_096_

# Resample images with different slice numbers 

# Binarize masks

In [ ]:
data_dir = '/Volumes/data/Bulbus/resampled'
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith("ob_left.nii.gz") or file.endswith("ob_right.nii.gz") :
            fname = file.split(".")[0]
            in_file = nib.load(os.path.join("resampled", file))
            in_data = in_file.get_fdata()
            in_data[in_data > 0.9] = 1
            in_data[in_data < 0.9] = 0
            out_fname = fname+str('_binarized')+".nii.gz"
            nib.save(nib.Nifti1Image(in_data, in_file.affine, header=in_file.header), os.path.join('binarized', out_fname))